<br>

<br><center><img src="https://storage.googleapis.com/kaggle-competitions/kaggle/34547/logos/header.png?t=2022-02-15-22-37-27" width=100%></center>

<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: #BF2C58; background-color: #ffffff;">🫁🫀 Explore Basic Submissions – HuBMAP+HPA – Organ Segmentation 🫀🫁</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

<br>

---

<br>

<center><div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">🛑 &nbsp; WARNING:</b><br><br><b>THIS IS A WORK IN PROGRESS</b><br>
</div></center>


<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">👏 &nbsp; IF YOU FORK THIS OR FIND THIS HELPFUL &nbsp; 👏</b><br><br><b style="font-size: 22px; color: darkorange">PLEASE UPVOTE!</b><br><br>This was a lot of work for me and while it may seem silly, it makes me feel appreciated when others like my work. 😅
</div></center>




In [ ]:
print("\n... IMPORTS STARTING ...\n")

print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_hub as tfhub; print(f"\t\t– TENSORFLOW HUB VERSION: {tfhub.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from pandarallel import pandarallel; pandarallel.initialize();
from sklearn.model_selection import GroupKFold, StratifiedKFold
from scipy.spatial import cKDTree

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import hashlib
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import json
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image, ImageEnhance
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2
    
print("\n\n... IMPORTS COMPLETE ...\n")

In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n\n")

# Open the training dataframe and display the initial dataframe
DATA_DIR = "/kaggle/input/hubmap-organ-segmentation"

# Open the training dataframe and display the initial dataframe
TRAIN_IMAGES_DIR = os.path.join(DATA_DIR, "train_images")
TRAIN_LABELS_DIR = os.path.join(DATA_DIR, "train_annotations")
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)

# Get all training images
all_train_images = glob(os.path.join(TRAIN_IMAGES_DIR, "*.tiff"), recursive=True)
all_train_labels = glob(os.path.join(TRAIN_LABELS_DIR, "*.json"), recursive=True)

print("\n... ORIGINAL TRAINING DATAFRAME... \n")
display(train_df)

TEST_IMAGES_DIR = os.path.join(DATA_DIR, "test_images")
TEST_CSV = os.path.join(DATA_DIR, "test.csv")
test_df = pd.read_csv(TEST_CSV)

SS_CSV   = os.path.join(DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)

# Get all testing images if there are any
all_test_images = glob(os.path.join(TEST_IMAGES_DIR, "*.tiff"), recursive=True)

print("\n\n\n... ORIGINAL SUBMISSION DATAFRAME... \n")
display(ss_df)

print("\n... BASIC DATA SETUP FINISHED ...\n\n")

In [ ]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
# modified from: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, shape, color=1):
    """ TBD
    
    Args:
        mask_rle (str): run-length as string formated (start length)
        shape (tuple of ints): (height,width) of array to return 
    
    Returns: 
        Mask (np.array)
            - 1 indicating mask
            - 0 indicating background

    """
    # Split the string by space, then convert it into a integer array
    s = np.array(mask_rle.split(), dtype=int)

    # Every even value is the start, every odd value is the "run" length
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths

    # The image image is actually flattened since RLE is a 1D "run"
    if len(shape)==3:
        h, w, d = shape
        img = np.zeros((h * w, d), dtype=np.float32)
    else:
        h, w = shape
        img = np.zeros((h * w,), dtype=np.float32)

    # The color here is actually just any integer you want!
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
        
    # Don't forget to change the image back to the original shape
    return img.reshape(shape).T

# https://www.kaggle.com/namgalielei/which-reshape-is-used-in-rle
def rle_decode_top_to_bot_first(mask_rle, shape):
    """ TBD
    
    Args:
        mask_rle (str): run-length as string formated (start length)
        shape (tuple of ints): (height,width) of array to return 
    
    Returns:
        Mask (np.array)
            - 1 indicating mask
            - 0 indicating background

    """
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape((shape[1], shape[0]), order='F').T  # Reshape from top -> bottom first

# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    """ TBD
    
    Args:
        img (np.array): 
            - 1 indicating mask
            - 0 indicating background
    
    Returns: 
        run length as string formated
    """
    
    img = img.T
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]

def load_json_to_dict(json_path):
    """ tbd """
    with open(json_path) as json_file:
        data = json.load(json_file)
    return data

def tf_decode_tiff(img_path, to_numpy=False, to_rgb=False):
    img = tf.io.read_file(img_path)
    img = tfio.experimental.image.decode_tiff(img)
    
    # Optionals
    if to_rgb: img = tfio.experimental.color.rgba_to_rgb(img)
    if to_numpy: img = img.numpy()
        
    return img

In [ ]:
# Probing
# if (test_df.pixel_size>1.0).sum()>0:
#     raise ValueError()

In [ ]:
SUBMISSION_STYLE = "smart_circle_ones" # [all_zeros', 'all_ones', 'all_random', 'smart_circle_ones']
SMART_CIRCLE_FRAC = 0.8

if SUBMISSION_STYLE=="all_zeros":
    ss_df["rle"] = ""
elif SUBMISSION_STYLE=="all_ones":
    ss_df["rle"] = "1 "+(test_df["img_width"]*test_df["img_height"]).astype(str)
elif SUBMISSION_STYLE=="all_random":
    ss_df["rle"] = test_df.apply(lambda row: rle_encode(np.where(np.random.random((row["img_width"], row["img_height"]))>0.5, 1.0, 0.0)), axis=1)

# All ones in interior circle all zeros outside (mimic tissue sample)
#   - To construct our circle:
#       - we imagine a square (img_width by img_height)
#       - we then draw a circle originating at the image centre (img_width//2, img_height//2) w/ a radius of some % (let's say 80)
elif SUBMISSION_STYLE=="smart_circle_ones":
    rles = []
    for i, (img_w, img_h) in enumerate(zip(test_df["img_width"], test_df["img_height"])):
        tmp_img = np.zeros((img_w, img_h))
        tmp_img = cv2.circle(tmp_img, (int(np.round(img_w/2)), int(np.round(img_h/2))), int(np.round((img_w/2)*SMART_CIRCLE_FRAC)), 1, -1)
        rles.append(rle_encode(tmp_img))
        if i==0:
            plt.figure(figsize=(10,10))
            plt.imshow(rle_decode(rles[0], (img_w, img_h)))
            plt.title("Example of Smart Circle Mask", fontweight="bold")
            plt.show()
    ss_df["rle"] = rles
else:
    raise NotImplementedError()

In [ ]:
# else:
#     # 1. Remove broken masks
#     remove_ids = ["case7_day0", "case81_day30"]
#     for _id in remove_ids:
#         train_df = train_df[~train_df.id.str.contains(_id)].reset_index(drop=True)

#     # 2. Get existing training data mappings to align with known metadata
#     slice_px_map = {}
#     for _, row in train_df.groupby(["slice_h", "slice_w", "px_spacing_h", "px_spacing_w", "slice_id"])[["lb_seg_rle", "sb_seg_rle", "st_seg_rle"]].first().reset_index().iterrows():
#         slice_px_map[f"{row['slice_h']}-{row['slice_w']}-{row['px_spacing_h']}-{row['px_spacing_w']}-{row['slice_id']}-large_bowel"] = row["lb_seg_rle"]
#         slice_px_map[f"{row['slice_h']}-{row['slice_w']}-{row['px_spacing_h']}-{row['px_spacing_w']}-{row['slice_id']}-small_bowel"] = row["sb_seg_rle"]
#         slice_px_map[f"{row['slice_h']}-{row['slice_w']}-{row['px_spacing_h']}-{row['px_spacing_w']}-{row['slice_id']}-stomach"] = row["st_seg_rle"]
    
#     # 3. Create a lookup and assign the prediction RLE
#     ss_df["ident"] = ss_df['slice_h'].astype(str)+"-"+ss_df['slice_w'].astype(str)+"-"+ss_df['px_spacing_h'].astype(str)+"-"+ss_df['px_spacing_w'].astype(str)+"-"+ss_df['slice_id'].astype(str)+"-"+ss_df["class"].astype(str)
#     ss_df["predicted"] = ss_df["ident"].map(slice_px_map)
#     ss_df["predicted"] = ss_df["predicted"].apply(lambda x: "" if x==None else x)

    
#     # 4. If we want some additional post processing we do that here
#     if SUBMISSION_STYLE=="smart_random":
        
#         # 4a. Remove RLE masks from areas where no mask should exist
#         ss_df = ss_df.apply(fix_empty_slices, axis=1)
        
#         # 4b. Remove non-contiguous masks that do not align with known patterns
#         ss_df["prev_predicted"] = ss_df.shift(3, fill_value="")["predicted"]
#         ss_df["next_predicted"] = ss_df.shift(-3, fill_value="")["predicted"]
#         ss_df["seg_isolated"] = ss_df.apply(is_isolated, axis=1)
#         ss_df = ss_df.apply(fix_nc_slices, axis=1)

In [ ]:
# 5. Reduce number of columns (just in case) and save
ss_df = ss_df[["id", "rle"]]
ss_df.to_csv("submission.csv", index=False)

display(ss_df)